In [1]:
# в этом ноутбуке я считаю профиты для лонг трейдов со скользящим стоплоссом.
# собираюсь использовать это как раметку данных.
import pandas as pd
import numpy as np
import pandas_ta as ta
import forecast
from quote_chart import create_chart_app
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from tsl import get_trailing_stop_loss_long_profits

fee_percent = 0.07

trailing_stop_loss_percent = 0.15

In [2]:
data = forecast.get_df('BTCUSD', '', '1T')

In [3]:
def get_volatility():
    df = data
    closes = df['close']
    return np.abs(np.log(closes/closes.shift(1)))
volatility = get_volatility()
data['volatility'] = volatility
data['volatility_ema'] = ta.ema(volatility, 20)
data['volatility_ema1k'] = ta.ema(volatility, 10*1000)
data['volatility_multiplier'] = (1+data['volatility_ema']/data['volatility_ema1k']) **1.5

In [4]:
dc = ta.donchian(data['high'], data['low'], lower_length=20, upper_length=20)
dclow, dcmid, dcup = dc[dc.columns[0]], dc[dc.columns[1]], dc[dc.columns[2]]
data['dclow'] = dclow

In [8]:
(profits, stop_loss_prices, maximums) = get_trailing_stop_loss_long_profits(data)
data['profit'] = profits
data['stop_loss_price'] = stop_loss_prices
data['maximum'] = maximums

0
2
3
9
10
11
12
16
17
18
19
20
29


IndexError: list index out of range

In [ ]:
len(dclow)

568800

In [ ]:
resampled_df = data.copy()

# called when a period button is pressed under the plot.
def on_period_change(button_id):
    global resampled_df, selected_period
    if button_id == '':
        return
    selected_period = button_id
    resampled_df = data.resample(selected_period).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'volatility_multiplier': 'last',
        'profit': 'first',
        'stop_loss_price': 'last',
        'maximum': 'last',
        'dclow': 'last',
    })
    resampled_df = resampled_df.dropna()

def create_figure(x0, x1):
    if x0 is not None:
        print(x0)
        df = resampled_df[x0:x1]
    else:
        df = resampled_df[-100:]#
        # xx0 = datetime.strptime('2024-05-20 00:03:44.250000', '%Y-%m-%d %H:%M:%S.%f')
        # xx1 = datetime.strptime('2024-05-21 02:03:44.250000', '%Y-%m-%d %H:%M:%S.%f')
        # df = resampled_df[xx0:xx1]
        
    # define multiple panes. The top pane will be for the main price chart with candles. The second pane is for volumes.
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.01,
                row_heights=[0.8, 0.2],
                specs=[[{"secondary_y": True}], [{"secondary_y": True}]])
    # plot the main chart with price candles.
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name='ohlc'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['open'] + df['profit'], mode='lines', line=dict(color='black'), name='open+profit'), row=1, col=1)
    # fig.add_trace(go.Scatter(x=df.index, y=df['open'] + df['profit'] - (df['maximum'] - df['close']), mode='lines', line=dict(color='orange'), name='discounted profit'), row=1, col=1)
    # fig.add_trace(go.Scatter(x=df.index, y=df['open'] * (1 + 0.01*fee_percent*2), mode='lines', line=dict(color='blue'), name='fee'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['stop_loss_price'], mode='lines', line=dict(color='red'), name='stop_loss_pirce'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['maximum'], mode='lines', line=dict(color='green'), name='maximum'), row=1, col=1)
    # fig.add_trace(go.Scatter(x=df.index, y=df['dclow'], mode='lines', line=dict(color='purple'), name='dclow'), row=1, col=1)
    fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume'), row=2, col=1)
    # fig.add_trace(go.Scatter(x=df.index, y=df['open'] + df['profit'] - (df['maximum'] - df['close']) - df['open'] * (1 + 0.01*fee_percent*2), mode='lines', line=dict(color='green'), name='dis prof wo fee'), row=1, col=1, secondary_y=True)
    # set the default dragmode to pan, remove the range slider because i use zoom/pan instead of it.
    fig.update_layout(
        dragmode='pan',
        xaxis_rangeslider_visible=False,
        width=1200, # px
        height=600,
        margin=dict(l=50, r=0, t=0, b=0),
        yaxis=dict(side='right'),
        yaxis2=dict(side='left'),
        yaxis3=dict(side='right'),
        yaxis4=dict(side='left'),
        yaxis5=dict(side='right'),
        yaxis6=dict(side='left'),
        )
    fig.update_xaxes(
        ticklabelposition="outside right",  # keep labels on the right so that they don't affect margin-left.
    )
    return fig

app = create_chart_app(create_figure, on_period_change, period_buttons=None, debug=False)
app.run_server(debug=True)